In [0]:
import requests
import json
from datetime import datetime
import time

data_hoje = datetime.now().strftime("%Y-%m-%d")
all_data = []
page = 1
max_retries = 3

print(f"🚀 Iniciando extração: {data_hoje}")

while True:
    url = f"https://api.openbrewerydb.org/v1/breweries?page={page}&per_page=200"

    success = False
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            success = True
            break

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Erro na página {page} (Tentativa {attempt + 1}): {e}")
            time.sleep(2)

    if not success:
        print(f"❌ Falha crítica ao acessar a página {page}. Abortando.")
        break

    if not data:
        break

    all_data.extend(data)
    print(f"✅ Página {page} processada. Total acumulado: {len(all_data)}")
    page += 1

if all_data:
    nome_arquivo = f"breweries_raw_{data_hoje}.json"
    try:
        with open(nome_arquivo, "w") as f:
            json.dump(all_data, f)

        from google.colab import files
        files.download(nome_arquivo)
        print(f"🏁 Extração concluída! {len(all_data)} registros salvos.")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo: {e}")
else:
    print("⚠️ Nenhum dado foi extraído.")